Pororo Abstractive Summarization

Public Score ROUGE-N에 대한 F1 SCORE : 0.38461862	0.27563649	0.3336278

In [ ]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

In [ ]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

In [ ]:
import numpy as np
import pandas as pd

import json
from pandas.io.json import json_normalize

import re
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os

DIR = "/content/drive/MyDrive/DACON_LG/data/"
TRAIN_SOURCE = os.path.join(DIR, "train.json")
TEST_SOURCE = os.path.join(DIR, "test.json")

In [ ]:
with open(TRAIN_SOURCE) as f:
    TRAIN_DATA = json.loads(f.read())
    
with open(TEST_SOURCE) as f:
    TEST_DATA = json.loads(f.read())

In [ ]:
train = pd.DataFrame(columns=['uid', 'title', 'region', 'context', 'summary'])
uid = 1000
for data in TRAIN_DATA:
    for agenda in data['context'].keys():
        context = ''
        for line in data['context'][agenda]:
            context += data['context'][agenda][line]
            context += ' '
        train.loc[uid, 'uid'] = uid
        train.loc[uid, 'title'] = data['title']
        train.loc[uid, 'region'] = data['region']
        train.loc[uid, 'context'] = context[:-1]
        train.loc[uid, 'summary'] = data['label'][agenda]['summary']
        uid += 1

test = pd.DataFrame(columns=['uid', 'title', 'region', 'context'])
uid = 2000
for data in TEST_DATA:
    for agenda in data['context'].keys():
        context = ''
        for line in data['context'][agenda]:
            context += data['context'][agenda][line]
            context += ' '
        test.loc[uid, 'uid'] = uid
        test.loc[uid, 'title'] = data['title']
        test.loc[uid, 'region'] = data['region']
        test.loc[uid, 'context'] = context[:-1]
        uid += 1

In [ ]:
train['total'] = train.title + ' ' + train.region + ' ' + train.context
test['total'] = test.title + ' ' + test.region + ' ' + test.context

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
testcopy = test['total']

In [ ]:
testcopy

In [ ]:
!pip install --upgrade pororo

In [ ]:
!pip install kollocate

In [ ]:
!pip install koparadigm

In [ ]:
!pip install g2pk

In [ ]:
testcopy_total_list = testcopy.tolist()

In [ ]:
testcopy_total_list[0]

In [ ]:
'''
running this cell, error occurs
Change 'generations_utils.py' line 1835 from 
# next_indices = (next_tokens / vocab_size).long()
-> next_indices = next_tokens.true_divide(vocab_size).long()
'''
from pororo import Pororo

summ_abs = Pororo(task="summarization", model="abstractive", lang="ko")

abs_result = []

for i in range(0,len(testcopy_total_list)):
  abs_result.append(summ_abs(testcopy_total_list[i]))

In [ ]:
abs_result[:10]

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/DACON_LG/data/sample_submission.csv')

In [ ]:
submission

In [ ]:
abs_result = pd.Series(abs_result)

In [ ]:
submission['summary'] = abs_result

In [ ]:
submission

,uid,summary
0,id_2000-AGENDA_1,군의회 제235회 제1차 정례회 제1차 본회의에서는 의회사무과장이 보고드리겠습니다.
1,id_2000-AGENDA_2,"군의회 제235회 제1차 정례회 회기결정건에서 환경분야 현지확인 특위 구성의 건, ..."
2,id_2000-AGENDA_3,"제2 의사일정 제2항, 회의록 서명의원 선출의 건에서 조천희 의원님, 손달섭 의원..."
3,id_2000-AGENDA_4,"의사일정 제3항, 예산결산특별위원회 구성의 건을 상정하여, 제가 제의한 구성안에 대..."
4,id_2000-AGENDA_5,제235회 제1차 정례회 제1차 본회의를 맞이하여 음성군 환경분야 현지확인을 발의하...
...,...,...
501,id_2096-AGENDA_2,당초74회 음성군의회 임시회 회기연장은 농협 축산물공판장 이전관련 특별위원회에서 위...
502,id_2096-AGENDA_3,본태완 위원장님께서는 지난 10월 10일 제1차 본회의시 농협 축산물공판장 이전 ...
503,id_2096-AGENDA_4,문화공보과장은 문화공보과에서 제출한 대소도서관 관리운영조례안을 상정하였는데 그 내용...
504,id_2096-AGENDA_5,보건기구에 의하면 제4기 음성군 지역보건 의료계획은 지역보건법 제3조 및 동법시행령...


In [ ]:
submission.to_csv('/content/drive/MyDrive/DACON_LG/data/1012_dacon_pororo_strong_abstract.csv', index=False)